# Regular 3d fracture network

### Problem description

Three-dimensional problem, based on the test case 4.1 from the benchmark study [Flemisch 2017]. The geometry is slightly modified to allow a full 3d solution field. Both low permeable and high permeable fractures with respect to the rock matrix are considered.

### Data of the problem

The domain is $\Omega = (0, 1)^3$ and 9 fractures are present, see the `file geiger_3d.csv` for the coordinates. A graphical representation is given by<br>
<img src="geiger_network_0.png" width="50%"/>

Given a reference aperture $\varepsilon_{ref}$, we set for each dimension $d$ the aperture as $\varepsilon^d = \varepsilon_{ref}^{3-d}$.

We define the pressure boundary $\partial \Omega_{p} = \{ (x, y, z) \in \partial \Omega: x, y, z > 0.75\}$, we introduce also the non-homogeneous flux boundary $\partial \Omega_{flux} = \{ (x, y, z) \in \partial \Omega: x, y, z < 0.5\}$. On $\partial \Omega_{p}$ we impose unitary pressure condition, while on $\partial \Omega_{flux}$ we impose $-1$ as flux boundary condition. On the rest of the boundary of $\Omega$ we impose homogeneous flux conditions. It is important to note that for the fractures we impose always the latter. The matrix permeability is given by DESCRIBE

Two test cases are considered
1. high permeable fractures, we set $k_{f, n} = k_{f, t} = 10^4$
2. low permeable fractures, we set $k_{f, n} = k_{f, t} = 10^{-4}$

In the code it is possible to select the test by setting the following line to 1 or 2.

In [1]:
test_case = 1
assert test_case == 1 or test_case == 2

We import the fracture network geometry and the domain, we create the grids and compute all the geometrical quantities.

In [2]:
import numpy as np
from porepy.fracs import importer

# geometrical tolerance
tol = 1e-8

# define the mesh size
h = 0.15
grid_kwargs = {}
grid_kwargs['mesh_size'] = {'mode': 'constant', 'value': h, 'bound_value': h, 'tol': tol}

# first line in the file is the domain boundary, the others the fractures
file_dfm = 'geiger_3d.csv'

# import the dfm and generete the grids
gb, domain = importer.dfm_3d_from_csv(file_dfm, tol, **grid_kwargs)
gb.compute_geometry()

Use existing intersections
Use existing decomposition
Using old version of mesh size determination
Gmsh processed file successfully


Grid creation completed. Elapsed time 0.16308355331420898


Created 1 3-d grids with 3773 cells
Created 9 2-d grids with 792 cells
Created 69 1-d grids with 90 cells
Created 27 0-d grids with 27 cells




To following lines are mandatory for dual vem approximation, however they will be removed in a newer version of the code.

In [3]:
%%capture
from porepy.grids.grid import FaceTag

internal_flag = FaceTag.FRACTURE
[g.remove_face_tag_if_tag(FaceTag.BOUNDARY, internal_flag) for g, _ in gb]

We introduce some of the data for the problem, others are hard coded in the class `DarcyModelData` in the file `geiger_3d_data.py` (mainly the definition of boundary conditions).

In [4]:
# select the permeability depending on the selected test case
if test_case == 1:
    kf = 1e4
else:
    kf = 1e-4
data_problem = {'domain': domain, 'tol': tol, 'aperture': 1e-4, 'km_low': 1e-1, 'km': 1, 'kf': kf}

Add the data to the grid bucket and the coupling permeability among objects of different dimensions.

In [5]:
from geiger_3d_data import DarcyModelData

gb.add_node_props(['is_tangential', 'problem', 'frac_num', 'low_zones'])
for g, d in gb:
    d['problem'] = DarcyModelData(g, d, **data_problem)
    d['is_tangential'] = True
    d['low_zones'] = d['problem'].low_zones()

    if g.dim == 2:
        d['frac_num'] = g.frac_num*np.ones(g.num_cells)
    else:
        d['frac_num'] = -1*np.ones(g.num_cells)        

# Assign coupling permeability, the aperture is read from the lower dimensional grid
gb.add_edge_prop('kn')
for e, d in gb.edges_props():
    g_l = gb.sorted_nodes_of_edge(e)[0]
    aperture = gb.node_prop(g_l, 'param').get_aperture()
    d['kn'] = data_problem['kf'] / aperture

Create the problem and solve it. Export the pressure and projected velocity for visualization.

In [6]:
from porepy.numerics import elliptic

problem = elliptic.DualEllipticModel(gb)
problem.solve()
problem.split()

problem.pressure('pressure')
problem.discharge('discharge')
problem.project_discharge('P0u')
problem.save(['pressure', 'P0u', 'frac_num', 'low_zones'])

To convert this script in pure python:

In [18]:
!jupyter nbconvert --to script geiger_3d.ipynb

[NbConvertApp] Converting notebook geiger_3d.ipynb to script
[NbConvertApp] Writing 599 bytes to geiger_3d.py
